In [ ]:
#importing the necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('mail_data.csv') #Importing the data-set

In [ ]:
print(df)

In [ ]:
data = df.where((pd.notnull(df)), '')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.shape #To see the number of rows & columns

In [ ]:
data.loc[data['Category'] == 'spam', 'Category',] = 0
data.loc[data['Category'] == 'ham', 'Category',] = 1

In [ ]:
X= data['Message']
Y= data['Category']

In [ ]:
print(X)

In [ ]:
print(Y)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state = 3)
#0.2 means 80% training & 20% testing
#Random is used in ML to get consistent result

In [ ]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

In [ ]:
print(Y.shape)
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
# Initialize the TfidfVectorizer
feature_extraction = TfidfVectorizer(min_df=1, stop_words='english', lowercase=True)
#stop_words is here used to ignore those words which can be saely ignored wihtout changing the meaning of the sentene (eg: the, have, an, etc.)

# Transform the training and test data
X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

# Convert Y labels to integer
Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

In [ ]:
print(X_train)

In [ ]:
print(X_train_features)

In [ ]:
model = LogisticRegression()

In [ ]:
#training the logisticregression data with the training data
model.fit(X_train_features,Y_train) 

In [ ]:
# Evaluate model accuracy
prediction_on_training_data = model.predict(X_train_features)
accuracy_on_training_data = accuracy_score(Y_train, prediction_on_training_data)

prediction_on_test_data = model.predict(X_test_features)
accuracy_on_test_data = accuracy_score(Y_test, prediction_on_test_data)

In [ ]:
print(f"Accuracy on Training Data: {accuracy_on_training_data * 100:.2f}%")
print(f"Accuracy on Test Data: {accuracy_on_test_data * 100:.2f}%")

In [ ]:
# Test the model with custom input
input_your_mail = [
    "This is the 2nd time we have tried to contact you. You have won the Rs 200000 prize. To claim is easy, just call"
]

# Transform the input text
input_data_features = feature_extraction.transform(input_your_mail)

# Predict
prediction = model.predict(input_data_features)

# Output the result
if prediction[0] == 1:
    print('Ham mail')
else:
    print('Spam mail')

In [ ]:
#Exploring the Dataset

In [ ]:
# Distribution of Spam vs. Ham
category_counts = data['Category'].value_counts()
print(category_counts)

# Average message length by category
data['Message_Length'] = data['Message'].apply(len)
avg_length = data.groupby('Category')['Message_Length'].mean()
print(avg_length)

In [ ]:
#Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Bar plot for category distribution
category_counts.plot(kind='bar', color=['green', 'red'])
plt.title('Spam vs Ham Distribution')
plt.xlabel('Category')
plt.ylabel('Count')
plt.xticks(ticks=[0, 1], labels=['Ham', 'Spam'], rotation=0)
plt.show()

# Boxplot for message length
sns.boxplot(x='Category', y='Message_Length', data=data)
plt.title('Message Length by Category')
plt.show()

In [ ]:
#Keyword Analysis 
#Identifying the most frequently used words in spam vs. ham messages using TfidfVectorizer or Counter

In [ ]:
from collections import Counter

# Create a function to extract common words
def common_words(messages, top_n=10):
    all_words = ' '.join(messages).split()
    return Counter(all_words).most_common(top_n)

# Get top words in spam and ham
spam_words = common_words(data[data['Category'] == 0]['Message'])
ham_words = common_words(data[data['Category'] == 1]['Message'])

print("Top words in Spam:", spam_words)
print("Top words in Ham:", ham_words)

In [ ]:
from wordcloud import WordCloud

spam_text = ' '.join(data[data['Category'] == 0]['Message'])
ham_text = ' '.join(data[data['Category'] == 1]['Message'])

spam_wordcloud = WordCloud(width=800, height=400, background_color='red').generate(spam_text)
ham_wordcloud = WordCloud(width=800, height=400, background_color='green').generate(ham_text)

# Spam WordCloud
plt.imshow(spam_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Spam WordCloud")
plt.show()

# Ham WordCloud
plt.imshow(ham_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Ham WordCloud")
plt.show()

In [ ]:
#Analyzing Message Length Distribution
#Examine how message length varies across spam and ham. Longer messages could have a pattern worth analyzing.

In [ ]:
# Plot histogram of message lengths by category
import matplotlib.pyplot as plt
import seaborn as sns

sns.histplot(data=data, x='Message_Length', hue='Category', bins=30, kde=True, palette='coolwarm')
plt.title('Message Length Distribution by Category')
plt.xlabel('Message Length')
plt.ylabel('Frequency')
plt.show()

In [ ]:
#Character-Based Features
#Calculate and compare features like:
#Number of uppercase letters (often used in spam for emphasis).
#Special character count (e.g., $, !, #).

In [ ]:
# Create new features
data['Uppercase_Count'] = data['Message'].apply(lambda x: sum(1 for char in x if char.isupper()))
data['Special_Char_Count'] = data['Message'].apply(lambda x: sum(1 for char in x if char in '!@#$%^&*'))

# Compare across categories
print(data.groupby('Category')[['Uppercase_Count', 'Special_Char_Count']].mean())


In [ ]:
# Boxplot for uppercase letter counts
sns.boxplot(x='Category', y='Uppercase_Count', data=data)
plt.title('Uppercase Letter Count by Category')
plt.show()

# Boxplot for special character counts
sns.boxplot(x='Category', y='Special_Char_Count', data=data)
plt.title('Special Character Count by Category')
plt.show()


In [ ]:
#Model Evaluation Enhancements

In [ ]:
#a. Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Confusion matrix
cm = confusion_matrix(Y_test, prediction_on_test_data)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Spam', 'Ham'])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
#b. Precision, Recall, F1-Score

In [ ]:
from sklearn.metrics import classification_report

# Classification report
report = classification_report(Y_test, prediction_on_test_data, target_names=['Spam', 'Ham'])
print(report)


In [ ]:
#Feature Importance Analysis
#Identifying which features (words) contribute the most to spam classification using the coefficients from the Logistic Regression model.

In [ ]:
# Get feature importance from the model
feature_names = feature_extraction.get_feature_names_out()
coefficients = model.coef_[0]

# Combine feature names and their importance scores
feature_importance = pd.DataFrame({'Feature': feature_names, 'Importance': coefficients})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

# Top 10 positive and negative features
print("Top 10 Spam Indicators:\n", feature_importance.head(10))
print("Top 10 Ham Indicators:\n", feature_importance.tail(10))


In [ ]:
# Bar plot for top spam indicators
top_spam_features = feature_importance.head(10)
top_spam_features.plot(kind='bar', x='Feature', y='Importance', legend=False, color='red')
plt.title('Top Words Indicating Spam')
plt.show()


In [ ]:
#Experimenting with Other Models

In [ ]:
#a. Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train_features, Y_train)

# Evaluate
rf_prediction = rf_model.predict(X_test_features)
rf_accuracy = accuracy_score(Y_test, rf_prediction)
print(f"Random Forest Accuracy: {rf_accuracy * 100:.2f}%")


In [ ]:
#b. Support Vector Machine (SVM)

In [ ]:
from sklearn.svm import SVC

# Train SVM
svm_model = SVC()
svm_model.fit(X_train_features, Y_train)

# Evaluate
svm_prediction = svm_model.predict(X_test_features)
svm_accuracy = accuracy_score(Y_test, svm_prediction)
print(f"SVM Accuracy: {svm_accuracy * 100:.2f}%")


In [ ]:
#Comparing Model Performances

In [ ]:
# Collect accuracy scores
model_accuracies = {
    "Logistic Regression": accuracy_on_test_data,
    "Random Forest": rf_accuracy,
    "SVM": svm_accuracy,
}

# Print model performances
print("Model Performance Comparison:")
for model, accuracy in model_accuracies.items():
    print(f"{model}: {accuracy * 100:.2f}%")

# Bar plot for model comparison
import matplotlib.pyplot as plt

plt.bar(model_accuracies.keys(), [v * 100 for v in model_accuracies.values()], color=['blue', 'green', 'orange'])
plt.ylabel('Accuracy (%)')
plt.title('Model Performance Comparison')
plt.show()


In [ ]:
#Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'C': [0.1, 1, 10], 
    'kernel': ['linear', 'rbf'], 
    'gamma': [0.1, 1, 10]
}

# Perform grid search
grid_search = GridSearchCV(SVC(), param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train_features, Y_train)

# Best parameters and accuracy
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

# Test with best model
best_svm = grid_search.best_estimator_
best_svm_accuracy = accuracy_score(Y_test, best_svm.predict(X_test_features))
print(f"Best SVM Test Accuracy: {best_svm_accuracy * 100:.2f}%")


In [ ]:
#Cross-Validation

In [ ]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation on the best model
scores = cross_val_score(best_svm, X_train_features, Y_train, cv=5, scoring='accuracy')
print(f"Cross-Validation Scores: {scores}")
print(f"Mean Cross-Validation Accuracy: {np.mean(scores) * 100:.2f}%")


In [ ]:
#Cluster Analysis

In [ ]:
# using only the spam messages from the dataset. Transform the text data into numerical vectors using the TfidfVectorizer.
#Applying K-Means Clustering
#Using the KMeans algorithm to group similar spam messages into clusters.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas as pd

# Filter spam messages
spam_messages = data[data['Category'] == 0]['Message']

# Vectorize the spam messages using TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english', max_features=500)  # Limit features for better clustering
spam_vectors = tfidf.fit_transform(spam_messages)

# Apply K-Means clustering
num_clusters = 5  # Adjust based on your preference
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(spam_vectors)

# Get cluster labels
spam_messages_clustered = pd.DataFrame({
    'Message': spam_messages.values,
    'Cluster': kmeans.labels_
})

# Display sample messages for each cluster
for cluster in range(num_clusters):
    print(f"\nCluster {cluster} Messages:")
    print(spam_messages_clustered[spam_messages_clustered['Cluster'] == cluster].head(5)['Message'].to_string(index=False))


In [ ]:
#Visualizing the Clusters

In [ ]:
#a. Elbow Method for Optimal Clusters

In [ ]:
from sklearn.metrics import silhouette_score

# Determine the optimal number of clusters
inertia = []
silhouette_scores = []
cluster_range = range(2, 11)

for k in cluster_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(spam_vectors)
    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(spam_vectors, kmeans.labels_))

# Plot inertia and silhouette scores
plt.figure(figsize=(12, 5))

# Elbow plot
plt.subplot(1, 2, 1)
plt.plot(cluster_range, inertia, marker='o')
plt.title('Elbow Method: Inertia')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')

# Silhouette score plot
plt.subplot(1, 2, 2)
plt.plot(cluster_range, silhouette_scores, marker='o', color='red')
plt.title('Silhouette Score')
plt.xlabel('Number of Clusters')
plt.ylabel('Score')

plt.tight_layout()
plt.show()


In [ ]:
#b. Word Cloud for Clusters

In [ ]:
from wordcloud import WordCloud

for cluster in range(num_clusters):
    cluster_messages = spam_messages_clustered[spam_messages_clustered['Cluster'] == cluster]['Message']
    cluster_text = ' '.join(cluster_messages)
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(cluster_text)
    
    # Plot the word cloud
    plt.figure(figsize=(8, 4))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"Cluster {cluster} Word Cloud")
    plt.show()
